In [ ]:
# Import necessary packages

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn import neighbors
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from scipy.stats import norm
from operator import itemgetter
from sklearn.model_selection import cross_val_score

In [ ]:
# Import the datasets

dfHistorical = pd.read_csv('historical-mvps.csv')
dfCurrent = pd.read_csv('current-mvp-ladder.csv')

In [ ]:
# Preview the historical dataset

dfHistorical.head()

# Histogram of share

In [ ]:
plt.style.use('fivethirtyeight')

voteHist, ax = plt.subplots()

ax.hist(dfHistorical['Share'], bins = 16, edgecolor = 'white', linewidth = 3, normed = True, label = "Actual distribution")
voteHist.suptitle("Histogram of historical vote share", weight = 'bold', size = 18, y = 1.05)
ax.set_xlabel("Vote Share")
ax.set_ylabel("Frequency")

overall_mean = dfHistorical['Share'].mean()
overall_std = dfHistorical['Share'].std()

xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, overall_mean, overall_std)
ax.plot(x, p, 'k', linewidth=5, color='orange', label = "Normalized distribution")

ax.legend(loc='best', prop={'size': 9, "family": "Rockwell"})

voteHist.text(x = 0, y = -0.08,
    s = '____________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left')

voteHist.text(x = 0, y = -.14,
    s = 'dribbleanalytics.blogspot.com                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

voteHist.savefig('vote-hist.png', dpi = 400, bbox_inches = 'tight')

# Create models

In [ ]:
train, test = train_test_split(dfHistorical, test_size = 0.25, random_state = 10)

xtrain = train[['Team Wins', 'Overall Seed', 'PTS', 'TRB', 'AST', 'FG%', 'VORP', 'WS']]
ytrain = train[['Share']]

xtest = test[['Team Wins', 'Overall Seed', 'PTS', 'TRB', 'AST', 'FG%', 'VORP', 'WS']]
ytest = test[['Share']]

In [ ]:
# Create function that gives accuracy scores for each model

def scores(y, model):
    
    model.fit(xtrain, ytrain.values.ravel())
    y_pred = model.predict(xtest)
    
    print("Mean squared error: %.3f" % mean_squared_error(ytest, y_pred))
    print('R2 score: %.3f' % r2_score(ytest, y_pred))

    cvScore = cross_val_score(model, xtest, ytest.values.ravel(), cv = 3, scoring = 'r2')
    print("R2 cross validation score: %0.2f (+/- %0.2f)" % (cvScore.mean(), cvScore.std() * 2))
    
    for i in y_pred:
        y.append(i)

In [ ]:
svr = SVR(kernel='rbf', gamma=1e-4, C=100, epsilon = .1)

y_svr = []

scores(y_svr, svr)

In [ ]:
rf = RandomForestRegressor(random_state = 9, n_estimators = 100, criterion = 'mse')

y_rf = []

scores(y_rf, rf)

In [ ]:
knn = neighbors.KNeighborsRegressor(n_neighbors = 7, weights = 'uniform')

y_knn = []

scores(y_knn, knn)

In [ ]:
dnn = MLPRegressor(
    solver='lbfgs',
    hidden_layer_sizes=100,
    max_iter=10000,
    random_state=987654321,
    activation='identity',
    learning_rate ='invscaling')

y_dnn = []

scores(y_dnn, dnn)

# Standardized residuals test

In [ ]:
#Create function that prepares the standardized residuals

def residuals(x, y):
    
    resid = [i for i in (ytest['Share'] - x)]
    ssr = [i ** 2 for i in resid]
    
    ssrSum = 0
    for i in ssr:
        ssrSum += i
        
    standResid = []
    for i in resid:
        standResid.append(i / ((ssrSum / (ytest.shape[0] - 2)) ** (1/2)))
    
    for i in standResid:
        y.append(i)

In [ ]:
svrResid = []
rfResid = []
knnResid = []
dnnResid = []

residuals(y_svr, svrResid)
residuals(y_rf, rfResid)
residuals(y_knn, knnResid)
residuals(y_dnn, dnnResid)

In [ ]:
def outliers(x):
    
    npList = np.array(x)
    stdev = np.std(npList)
    mean = np.mean(npList)

    outliers = 0
    for i in x:
        if i < mean - 2 * stdev:
            outliers += 1
        elif i > mean + 2 * stdev:
            outliers += 1

    outlierPercent = 1 - outliers / ytest.shape[0]
    outlierString = "{:.3%}".format(outlierPercent)
    
    return outlierString

In [ ]:
plt.style.use('fivethirtyeight')

residFig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharey = 'row')

x = np.arange(ytest.shape[0])
ax1.xaxis.set_visible(False)
ax2.xaxis.set_visible(False)
ax3.xaxis.set_visible(False)
ax4.xaxis.set_visible(False)

ax1.scatter(x, svrResid)
ax1.axhline(y = np.mean(svrResid), label = "Average")
ax1.axhline(y = np.mean(svrResid) - 2 * np.std(svrResid), color = 'orange')
ax1.axhline(y = np.mean(svrResid) + 2 * np.std(svrResid), label = "2 stdev from mean", color = 'orange')
ax1.set_title("SVM: %s" % outliers(svrResid), size = 18, x = .485, ha = 'center')

ax2.scatter(x, rfResid)
ax2.axhline(y = np.mean(rfResid))
ax2.axhline(y = np.mean(rfResid) - 2 * np.std(rfResid), color = 'orange')
ax2.axhline(y = np.mean(rfResid) + 2 * np.std(rfResid), color = 'orange')
ax2.set_title("RF: %s" % outliers(rfResid), size = 18, x = .485, ha = 'center')

ax3.scatter(x, knnResid)
ax3.axhline(y = np.mean(knnResid))
ax3.axhline(y = np.mean(knnResid) - 2 * np.std(knnResid), color = 'orange')
ax3.axhline(y = np.mean(knnResid) + 2 * np.std(knnResid), color = 'orange')
ax3.set_title("k-NN: %s" % outliers(knnResid), size = 18, x = .485, ha = 'center')

ax4.scatter(x, dnnResid)
ax4.axhline(y = np.mean(dnnResid))
ax4.axhline(y = np.mean(dnnResid) - 2 * np.std(dnnResid), color = 'orange')
ax4.axhline(y = np.mean(dnnResid) + 2 * np.std(dnnResid), color = 'orange')
ax4.set_title("DNN: %s" % outliers(dnnResid), size = 18, x = .485, ha = 'center')

residFig.legend(loc = (.25, .88), ncol=2, prop={'size': 12, "family": "Rockwell"})

residFig.suptitle("Standardized Residuals", weight = 'bold', size = 18, y = 1.08)

ax1.yaxis.set_ticks([-2.5, 0, 2.5])
ax3.yaxis.set_ticks([-2.5, 0, 2.5])

residFig.text(x = 0, y = 0,
    s = '________________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left')

residFig.text(x = 0, y = -.06,
    s = 'dribbleanalytics.blogspot.com                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

residFig.savefig('resid-fig.png', dpi = 400, bbox_inches = 'tight')

# Q-Q plot

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
plt.style.use('fivethirtyeight')

import scipy.stats as stats

qqplot = plt.figure()

ax1 = qqplot.add_subplot(221)
stats.probplot(svrResid, dist="norm", plot=plt)
ax1.set_xlabel("")
ax1.set_xticklabels([])
ax1.set_ylabel("")
ax1.set_title("SVM")

ax2 = qqplot.add_subplot(222)
stats.probplot(rfResid, dist="norm", plot=plt)
ax2.set_xlabel("")
ax2.set_xticklabels([])
ax2.set_ylabel("")
ax2.set_title("RF")
ax2.set_xticklabels([])
ax2.set_yticklabels([])

ax3 = qqplot.add_subplot(223)
stats.probplot(knnResid, dist="norm", plot=plt)
ax3.set_xlabel("")
ax3.set_ylabel("")
ax3.set_title("KNN")

ax4 = qqplot.add_subplot(224)
stats.probplot(dnnResid, dist = "norm", plot = plt)
ax4.set_xlabel("")
ax4.set_ylabel("")
ax4.set_title("DNN")
ax4.set_yticklabels([])

qqplot.text(0.5, -0.02, 'Theoretical Quantiles', ha='center', va='center', size = 18)
qqplot.text(0.01, 0.5, 'Ordered Values', ha='center', va='center', rotation='vertical', size = 18)

qqplot.text(x = 0, y = -0.05,
    s = '_______________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left')

qqplot.text(x = 0, y = -.1,
    s = 'dribbleanalytics.blogspot.com                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

qqplot.savefig('qqplot.png', dpi = 400, bbox_inches = 'tight')

# Shapiro-Wilk test

In [ ]:
from scipy.stats import shapiro

print(shapiro(svrResid))
print(shapiro(rfResid))
print(shapiro(knnResid))
print(shapiro(dnnResid))

# Durbin-Watson test 

In [ ]:
from statsmodels.stats import stattools as stools

print(stools.durbin_watson(svrResid))
print(stools.durbin_watson(rfResid))
print(stools.durbin_watson(knnResid))
print(stools.durbin_watson(dnnResid))

# Learning curve for MSE

In [ ]:
from sklearn.model_selection import learning_curve

train_sizes = [1, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250]


features = ['Team Wins', 'Overall Seed', 'PTS', 'TRB', 'AST', 'FG%', 'VORP', 'WS']
target = 'Share'

train_sizesSVR, train_scoresSVR, validation_scoresSVR = learning_curve(estimator = SVR(
    kernel='rbf', gamma=1e-4, C=100, epsilon = .1),
    X = dfHistorical[features], y = dfHistorical[target], train_sizes = train_sizes, cv = 3,
    scoring = 'neg_mean_squared_error')

In [ ]:
train_sizesRF, train_scoresRF, validation_scoresRF = learning_curve(estimator = RandomForestRegressor(
    random_state = 9, n_estimators = 100, criterion = 'mse'),
    X = dfHistorical[features], y = dfHistorical[target], train_sizes = train_sizes, cv = 3,
    scoring = 'neg_mean_squared_error')

In [ ]:
train_sizes1 = [7, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250]

train_sizesKNN, train_scoresKNN, validation_scoresKNN = learning_curve(estimator = neighbors.KNeighborsRegressor(
    n_neighbors = 7, weights = 'uniform'),
    X = dfHistorical[features], y = dfHistorical[target], train_sizes = train_sizes1, cv = 3,
    scoring = 'neg_mean_squared_error')

In [ ]:
train_sizesDNN, train_scoresDNN, validation_scoresDNN = learning_curve(estimator = MLPRegressor(
    solver='lbfgs',
    hidden_layer_sizes=100,
    max_iter=10000,
    random_state=987654321,
    activation='identity',
    learning_rate ='invscaling'),
    X = dfHistorical[features], y = dfHistorical[target], train_sizes = train_sizes, cv = 3,
    scoring = 'neg_mean_squared_error')

In [ ]:
train_scores_meanSVR = -train_scoresSVR.mean(axis = 1)
validation_scores_meanSVR = -validation_scoresSVR.mean(axis = 1)

train_scores_meanRF = -train_scoresRF.mean(axis = 1)
validation_scores_meanRF = -validation_scoresRF.mean(axis = 1)

train_scores_meanKNN = -train_scoresKNN.mean(axis = 1)
validation_scores_meanKNN = -validation_scoresKNN.mean(axis = 1)

train_scores_meanDNN = -train_scoresDNN.mean(axis = 1)
validation_scores_meanDNN = -validation_scoresDNN.mean(axis = 1)

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
plt.style.use('fivethirtyeight')

learningCurveMSE, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='col')

ax1.plot(train_sizesSVR, train_scores_meanSVR, label = 'Training set error')
ax1.plot(train_sizesSVR, validation_scores_meanSVR, label = 'Testing set error')
ax1.set_title("SVM")

ax2.plot(train_sizesRF, train_scores_meanRF)
ax2.plot(train_sizesRF, validation_scores_meanRF)
ax2.set_title("RF")

ax3.plot(train_sizesKNN, train_scores_meanKNN)
ax3.plot(train_sizesKNN, validation_scores_meanKNN)
ax3.set_title("KNN")

ax4.plot(train_sizesDNN, train_scores_meanDNN)
ax4.plot(train_sizesDNN, validation_scores_meanDNN)
ax4.set_title("DNN")

learningCurveMSE.suptitle("Learning Curve for MSE", weight = 'bold', size = 18, y = 1.08)
learningCurveMSE.legend(loc = (.21, .88), ncol=2, prop={'size': 12, "family": "Rockwell"})

learningCurveMSE.text(x = -0.02, y = 0,
    s = '_______________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left')

learningCurveMSE.text(x = -0.02, y = -.06,
    s = 'dribbleanalytics.blogspot.com                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

learningCurveMSE.savefig('learningCurveMSE.png', dpi = 400, bbox_inches = 'tight')

# Learning curve for R2

In [ ]:
train_sizes = [1, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250]

features = ['Team Wins', 'Overall Seed', 'PTS', 'TRB', 'AST', 'FG%', 'VORP', 'WS']
target = 'Share'

train_sizesSVR, train_scoresSVR, validation_scoresSVR = learning_curve(estimator = SVR(
    kernel='rbf', gamma=1e-4, C=100, epsilon = .1),
    X = dfHistorical[features], y = dfHistorical[target], train_sizes = train_sizes, cv = 3,
    scoring = 'r2')

In [ ]:
train_sizesRF, train_scoresRF, validation_scoresRF = learning_curve(estimator = RandomForestRegressor(
    random_state = 9, n_estimators = 100, criterion = 'mse'),
    X = dfHistorical[features], y = dfHistorical[target], train_sizes = train_sizes, cv = 3,
    scoring = 'r2')

In [ ]:
train_sizes1 = [7, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250]

train_sizesKNN, train_scoresKNN, validation_scoresKNN = learning_curve(estimator = neighbors.KNeighborsRegressor(
    n_neighbors = 7, weights = 'uniform'),
    X = dfHistorical[features], y = dfHistorical[target], train_sizes = train_sizes1, cv = 3,
    scoring = 'r2')

In [ ]:
train_sizesDNN, train_scoresDNN, validation_scoresDNN = learning_curve(estimator = MLPRegressor(
    solver='lbfgs',
    hidden_layer_sizes=100,
    max_iter=10000,
    random_state=987654321,
    activation='identity',
    learning_rate ='invscaling'),
    X = dfHistorical[features], y = dfHistorical[target], train_sizes = train_sizes, cv = 3,
    scoring = 'r2')

In [ ]:
train_scores_meanSVR = train_scoresSVR.mean(axis = 1)
validation_scores_meanSVR = validation_scoresSVR.mean(axis = 1)

train_scores_meanRF = train_scoresRF.mean(axis = 1)
validation_scores_meanRF = validation_scoresRF.mean(axis = 1)

train_scores_meanKNN = train_scoresKNN.mean(axis = 1)
validation_scores_meanKNN = validation_scoresKNN.mean(axis = 1)

train_scores_meanDNN = train_scoresDNN.mean(axis = 1)
validation_scores_meanDNN = validation_scoresDNN.mean(axis = 1)

In [ ]:
plt.style.use('fivethirtyeight')

learningCurveR2, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='col')

ax1.plot(train_sizesSVR, train_scores_meanSVR, label = 'Training set error')
ax1.plot(train_sizesSVR, validation_scores_meanSVR, label = 'Testing set error')
ax1.set_title("SVM")

ax2.plot(train_sizesRF, train_scores_meanRF)
ax2.plot(train_sizesRF, validation_scores_meanRF)
ax2.set_title("RF")

ax3.plot(train_sizesKNN, train_scores_meanKNN)
ax3.plot(train_sizesKNN, validation_scores_meanKNN)
ax3.set_title("KNN")

ax4.plot(train_sizesDNN, train_scores_meanDNN)
ax4.plot(train_sizesDNN, validation_scores_meanDNN)
ax4.set_title("DNN")

learningCurveR2.suptitle("Learning Curve for r-squared", weight = 'bold', size = 18, y = 1.08)
learningCurveR2.legend(loc = (.22, .88), ncol=2, prop={'size': 12, "family": "Rockwell"})

learningCurveR2.text(x = -0.02, y = 0,
    s = '_______________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left')

learningCurveR2.text(x = -0.02, y = -.06,
    s = 'dribbleanalytics.blogspot.com                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

learningCurveR2.savefig('learningCurveR2.png', dpi = 400, bbox_inches = 'tight')

# Prediction

In [ ]:
dfCurrentNames = dfCurrent.iloc[:, 1]
dfCurrentPredict = dfCurrent[['Team Wins', 'Overall Seed', 'PTS', 'TRB', 'AST', 'FG%', 'VORP', 'WS']]

dfCurrent.head()

## SVR 

In [ ]:
svrPredict = svr.predict(dfCurrentPredict)
svrPredict = svrPredict.tolist()

for (i, j) in zip(dfCurrentNames, svrPredict):
    print(i, j)

In [ ]:
svrListUnsorted = [[i, j] for i, j in zip(dfCurrentNames, svrPredict)]
svrDataUnsorted = [row[1] for row in svrListUnsorted]
svrList = sorted(svrListUnsorted, key = itemgetter(1), reverse = True)

svrData = [row[1] for row in svrList]
svrNames = [row[0] for row in svrList]
print(svrList)

x_svr = np.arange(len(svrData))

In [ ]:
plt.style.use('fivethirtyeight')

svr, ax = plt.subplots()

ax.bar(x_svr, svrData, width = .7, edgecolor = 'white', color = 'skyblue', linewidth = 4, label = 'Predicted')

labels = svrNames

rects = ax.patches
for rect, label in zip(rects, labels):
    if rect.get_x() > 7:
        ax.text(rect.get_x() + rect.get_width() / 1.75, rect.get_height() + .02, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')
    elif rect.get_x() <=7:
        height = .03
        ax.text(rect.get_x() + rect.get_width() / 1.75, height, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')
    
svr.suptitle("SVM predicted MVP share", weight = 'bold', size = 18, y = 1.005)
ax.set_title("NBA.com MVP ladder rank listed in parentheses", size = 14, fontname = 'Rockwell')
ax.xaxis.set_visible(False)
ax.set_ylabel("Vote Share")

svr.text(x = -0.02, y = 0.03,
    s = '_______________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left')

svr.text(x = -0.02, y = -.03,
    s = 'dribbleanalytics.blogspot.com                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

svr.savefig('svr.png', dpi = 400, bbox_inches = 'tight')

## RF

In [ ]:
rfPredict = rf.predict(dfCurrentPredict)
rfPredict = rfPredict.tolist()

for (i, j) in zip(dfCurrentNames, rfPredict):
    print(i, j)

In [ ]:
rfListUnsorted = [[i, j] for i, j in zip(dfCurrentNames, rfPredict)]
rfDataUnsorted = [row[1] for row in rfListUnsorted]
rfList = sorted(rfListUnsorted, key = itemgetter(1), reverse = True)

rfData = [row[1] for row in rfList]
rfNames = [row[0] for row in rfList]
print(rfList)

x_rf = np.arange(len(rfData))

In [ ]:
plt.style.use('fivethirtyeight')

rf, ax = plt.subplots()

ax.bar(x_rf, rfData, width = .7, edgecolor = 'white', color = 'skyblue', linewidth = 4, label = 'Predicted')

labels = rfNames

rects = ax.patches
for rect, label in zip(rects, labels):
    if rect.get_x() > 8:
        ax.text(rect.get_x() + rect.get_width() / 1.75, rect.get_height() + .02, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')
    elif rect.get_x() <= 8:
        height = .03
        ax.text(rect.get_x() + rect.get_width() / 1.75, height, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')
        
rf.suptitle("RF predicted MVP share", weight = 'bold', size = 18, y = 1.005)
ax.set_title("NBA.com MVP ladder rank listed in parentheses", size = 14, fontname = 'Rockwell')
ax.xaxis.set_visible(False)
ax.set_ylabel("Vote Share")

rf.text(x = -0.02, y = 0.03,
    s = '_______________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left')

rf.text(x = -0.02, y = -.03,
    s = 'dribbleanalytics.blogspot.com                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

rf.savefig('rf.png', dpi = 400, bbox_inches = 'tight')

## k-NN

In [ ]:
knnPredict = knn.predict(dfCurrentPredict)
knnPredict = knnPredict.tolist()

for (i, j) in zip(dfCurrentNames, knnPredict):
    print(i, j)

In [ ]:
knnListUnsorted = [[i, j] for i, j in zip(dfCurrentNames, knnPredict)]
knnDataUnsorted = [row[1] for row in knnListUnsorted]
knnList = sorted(knnListUnsorted, key = itemgetter(1), reverse = True)

knnData = [row[1] for row in knnList]
knnNames = [row[0] for row in knnList]
print(knnList)

x_knn = np.arange(len(knnData))

In [ ]:
plt.style.use('fivethirtyeight')

knn, ax = plt.subplots()

ax.bar(x_knn, knnData, width = .7, edgecolor = 'white', color = 'skyblue', linewidth = 4, label = 'Predicted')

labels = knnNames

rects = ax.patches
for rect, label in zip(rects, labels):
    if rect.get_x() > 6:
        ax.text(rect.get_x() + rect.get_width() / 1.75, rect.get_height() + .02, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')
    elif rect.get_x() <= 6:
        height = .03
        ax.text(rect.get_x() + rect.get_width() / 1.75, height, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')
        
knn.suptitle("KNN predicted MVP share", weight = 'bold', size = 18, y = 1.005)
ax.set_title("NBA.com MVP ladder rank listed in parentheses", size = 14, fontname = 'Rockwell')
ax.xaxis.set_visible(False)
ax.set_ylabel("Vote Share")

knn.text(x = -0.02, y = 0.03,
    s = '_______________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left')

knn.text(x = -0.02, y = -.03,
    s = 'dribbleanalytics.blogspot.com                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

knn.savefig('knn.png', dpi = 400, bbox_inches = 'tight')

## DNN

In [ ]:
dnnPredict = dnn.predict(dfCurrentPredict)
dnnPredict = dnnPredict.tolist()

for (i, j) in zip(dfCurrentNames, dnnPredict):
    print(i, j)

In [ ]:
dnnListUnsorted = [[i, j] for i, j in zip(dfCurrentNames, dnnPredict)]
dnnDataUnsorted = [row[1] for row in dnnListUnsorted]
dnnList = sorted(dnnListUnsorted, key = itemgetter(1), reverse = True)

dnnData = [row[1] for row in dnnList]
dnnNames = [row[0] for row in dnnList]
print(dnnList)

x_dnn = np.arange(len(dnnData))

In [ ]:
plt.style.use('fivethirtyeight')

dnn, ax = plt.subplots()

ax.bar(x_dnn, dnnData, width = .7, edgecolor = 'white', color = 'skyblue', linewidth = 4, label = 'Predicted')

labels = dnnNames

rects = ax.patches
for rect, label in zip(rects, labels):
    if rect.get_x() > 8:
        ax.text(rect.get_x() + rect.get_width() / 1.75, rect.get_height() + .02, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')
    elif rect.get_x() <= 8:
        height = .03
        ax.text(rect.get_x() + rect.get_width() / 1.75, height, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')
        
dnn.suptitle("DNN predicted MVP share", weight = 'bold', size = 18, y = 1.005)
ax.set_title("NBA.com MVP ladder rank listed in parentheses", size = 14, fontname = 'Rockwell')
ax.xaxis.set_visible(False)
ax.set_ylabel("Vote Share")

dnn.text(x = -0.02, y = 0.03,
    s = '_______________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left')

dnn.text(x = -0.02, y = -.03,
    s = 'dribbleanalytics.blogspot.com                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

dnn.savefig('dnn.png', dpi = 400, bbox_inches = 'tight')

## Average

In [ ]:
avgPredict = []

for (i, j, h, k) in zip(svrDataUnsorted, rfDataUnsorted, knnDataUnsorted, dnnDataUnsorted):
    avgPredict.append((i + j + h + k) / 4)

avgList = [[i, j] for i, j in zip(dfCurrentNames, avgPredict)]
avgList = sorted(avgList, key = itemgetter(1), reverse = True)

avgData = [row[1] for row in avgList]
avgNames = [row[0] for row in avgList]
print(avgList)

x_avg = np.arange(len(avgData))

In [ ]:
plt.style.use('fivethirtyeight')

avg, ax = plt.subplots()

ax.bar(x_avg, avgData, width = .7, edgecolor = 'white', color = 'skyblue', linewidth = 4, label = 'Predicted')

labels = avgNames

rects = ax.patches
for rect, label in zip(rects, labels):
    if rect.get_x() > 8:
        ax.text(rect.get_x() + rect.get_width() / 1.75, rect.get_height() + .02, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')
    elif rect.get_x() <= 8:
        height = .03
        ax.text(rect.get_x() + rect.get_width() / 1.75, height, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')
        
avg.suptitle("Average predicted MVP share", weight = 'bold', size = 18, y = 1.005)
ax.set_title("NBA.com MVP ladder rank listed in parentheses", size = 14, fontname = 'Rockwell')
ax.xaxis.set_visible(False)
ax.set_ylabel("Vote Share")

avg.text(x = -0.02, y = 0.03,
    s = '_______________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left')

avg.text(x = -0.02, y = -.03,
    s = 'dribbleanalytics.blogspot.com                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

avg.savefig('avg.png', dpi = 400, bbox_inches = 'tight')